In [1]:
import torch
import torch.nn as nn
import numpy as np
import scipy
import librosa
import youtube_dl
import os
import soundfile as sf
from IPython.display import Audio, display

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cpu


In [2]:
%cd ../models/Conditioned-Source-Separation-LaSAFT

/Users/arron/code/PlexedLive/mixs/models/Conditioned-Source-Separation-LaSAFT


In [3]:
from lasaft.source_separation.conditioned.cunet.models.dcun_tfc_gpocm_lasaft import DCUN_TFC_GPoCM_LaSAFT_Framework

args = {}

# FFT params
args['n_fft'] = 2048
args['hop_length'] = 1024
args['num_frame'] = 128

# SVS Framework
args['spec_type'] = 'complex'
args['spec_est_mode'] = 'mapping'

# Other Hyperparams
args['optimizer'] = 'adam'
args['lr'] = 0.001
args['dev_mode'] = False
args['train_loss'] = 'spec_mse'
args['val_loss'] = 'raw_l1'

# DenseNet Hyperparams

args ['n_blocks'] = 7
args ['input_channels'] = 4
args ['internal_channels'] = 24
args ['first_conv_activation'] = 'relu'
args ['last_activation'] = 'identity'
args ['t_down_layers'] = None
args ['f_down_layers'] = None
args ['tif_init_mode'] = None

# TFC_TDF Block's Hyperparams
args['n_internal_layers'] =5
args['kernel_size_t'] = 3
args['kernel_size_f'] = 3
args['tfc_tdf_activation'] = 'relu'
args['bn_factor'] = 16
args['min_bn_units'] = 16
args['tfc_tdf_bias'] = True
args['num_tdfs'] = 6
args['dk'] = 32

args['control_vector_type'] = 'embedding'
args['control_input_dim'] = 4
args['embedding_dim'] = 32
args['condition_to'] = 'decoder'

args['control_n_layer'] = 4
args['control_type'] = 'dense'
args['pocm_type'] = 'matmul'
args['pocm_norm'] = 'batch_norm'


model = DCUN_TFC_GPoCM_LaSAFT_Framework(**args)

In [4]:
model = model.load_from_checkpoint('pretrained/gpocm_lasaft.ckpt')

In [5]:
track_dir = '../../raw_data/musdb18/test/'
track_name = 'Tom McKenzie - Directions.stem.mp4'

audio, rate = librosa.load(f'{track_dir}{track_name}', mono=False, sr=44100)

In [7]:
audio.shape

(2, 7746297)

In [11]:
audio = audio[:, 20*rate:30*rate]

In [18]:
display(Audio(audio, rate=rate))

In [20]:
def separate_all(audio):
    '''
    you can add or remove what ever you want in this loop
    using the keys, vocals, drums, bass and other
    you can also combine them 
    '''
    
    print('vocals')
    separated = model.separate_track(audio.T, 'vocals') 
    vocals, sr=librosa.load('temp.wav', mono=False)

    print('drums')
    separated = model.separate_track(audio.T, 'drums') 
    drums, sr=librosa.load('temp.wav', mono=False)

    print('bass')
    separated = model.separate_track(audio.T, 'bass') 
    bass, sr=librosa.load('temp.wav', mono=False)

    print('other')
    separated = model.separate_track(audio.T, 'other') 
    other, sr=librosa.load('temp.wav', mono=False)
    
    print('mix')
    mix = [vocals, drums, bass, other]
    
    return mix
    
    
    


In [21]:
mix = separate_all(audio)

vocals
drums
bass
other
mix


In [24]:
mix = np.array(mix)

In [30]:
import musdb

SyntaxError: invalid syntax (<ipython-input-30-ad29e50a2145>, line 5)

In [31]:
DB = musdb.DB(root='../../raw_data/musdb18/samples', subsets='test')

In [53]:
audio = DB.tracks[0].audio

In [54]:
def separate_all(audio):
    '''
    you can add or remove what ever you want in this loop
    using the keys, vocals, drums, bass and other
    you can also combine them 
    '''
    
    print('vocals')
    separated = model.separate_track(audio, 'vocals') 
#     vocals, sr=librosa.load('temp.wav', mono=False)

#     print('drums')
#     separated = model.separate_track(audio.T, 'drums') 
#     drums, sr=librosa.load('temp.wav', mono=False)

#     print('bass')
#     separated = model.separate_track(audio.T, 'bass') 
#     bass, sr=librosa.load('temp.wav', mono=False)

#     print('other')
#     separated = model.separate_track(audio.T, 'other') 
#     other, sr=librosa.load('temp.wav', mono=False)
    
#     print('mix')
#     mix = [vocals, drums, bass, other]
    
    return separated

In [55]:
test = separate_all(audio)

vocals


KeyboardInterrupt: 

In [52]:
test.shape

(958464, 2)

In [ ]:
museval.eval_mus_dir(
    dataset=DB.tracks[0] # instance of musdb
    estimates_dir=...,  # path to estimate folder
    output_dir=...,  # set a folder to write eval json files
    subsets="test",
    is_wav=False
)